In [1]:
import json
import pandas as pd
from decimal import Decimal
from flask_cors import CORS, cross_origin
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify, request
import os
from datetime import timedelta, date,datetime
from os import remove
from flask_apscheduler import APScheduler

In [ ]:
class Config(object):
    SCHEDULER_API_ENABLED = True
    
scheduler = APScheduler()

def initData(data):
    lista = data.iloc[:,4:].columns
    dataRegion = pd.melt(data, id_vars =['Country/Region','Province/State'], value_vars =lista)

    groupData = dataRegion.groupby(['Country/Region','variable'])['value'].sum().reset_index()
    groupData['Date'] =pd.to_datetime(groupData.variable)
    groupData['MesDate'] = groupData['Date'].dt.strftime('%B')
    return groupData

app = Flask(__name__)

cors = CORS(app)
data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'  
data = pd.read_csv(data_url)
groupData = initData(data)

@scheduler.task('cron', id='cargarDatos', hour=10, minute=10)
def cargarDatos():
    data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'  
    global data
    global groupData
    data = pd.read_csv(data_url)
    print(data)
    groupData = initData(data)
    
    
app.config['CORS_HEADERS'] = 'Content-Type'




def estado(df, columna):
    for i in range(1,len(df)):
        if i-1 == 0:
            df.loc[i-1,'estado'] = False
        if (df[columna].loc[i] - df[columna].loc[i-1])>20:
            df.loc[i,'estado'] = True
        else:
            df.loc[i,'estado'] = False
    return df

def pais(pais,df):
    country = df[df["Country/Region"]==pais]
    country['Date'] =pd.to_datetime(country.variable)
    country = country.sort_values(by=['Date']).groupby(country['Date'].dt.strftime('%B')).max().sort_values(by=['value']) 
    total = country["value"].max()
    country["porcentaje"]= country["value"].astype("int").div(total)*100
    country["mes"] = country.index
    country.rename(columns = {'Date':'Fecha'}, inplace = True) 
    country = country.reset_index()
    country = estado(country,"porcentaje")
    labelY = list(country.loc[:,"Date"].astype("string"))
    lenRows = country.shape[0]
    valuesDf = country.loc[:,"value"]
    rowGrow = country[country["estado"]==True]
    totalMeses = pd.DataFrame()
    for index,row in rowGrow.iterrows():
        porpais = df[df["Country/Region"]==pais].reset_index()
        disMes = porpais[porpais['MesDate'] == row["mes"]]
        disMes = disMes.sort_values(by=['Date'])
        hotMonthY = list(disMes.loc[:,"variable"])
        hotMonth = list(disMes.loc[:,"MesDate"])
        hotMonthRows = disMes.shape[0]
        hotMonthDf = disMes.loc[:,"value"]
        dataFrameMes = pd.concat([pd.DataFrame(hotMonth).reset_index(drop=True),pd.DataFrame(hotMonthY).reset_index(drop=True), pd.DataFrame(hotMonthDf).reset_index(drop=True)], axis=1)
        totalMeses = pd.concat([totalMeses.reset_index(drop=True), dataFrameMes.reset_index(drop=True)], axis=0)
    totalDatos = pd.concat([pd.DataFrame(labelY).reset_index(drop=True), pd.DataFrame(valuesDf).reset_index(drop=True)], axis=1)
    totalDatos.columns = ['fechaTotal','datosTotal']
    totalMeses.columns = ['MesDate','fecha','datos',]
    totalDatos = totalDatos.reset_index(drop=True)
    totalMeses = totalMeses.reset_index(drop=True)
    return totalDatos,totalMeses,total

@app.route('/', methods=['GET'])
@cross_origin()
def getPrueba():
    return "Hola"

@app.route('/postDataCountry', methods=['POST'])
@cross_origin()
def getDataCountry():
    global groupData
    country = request.get_json()
    print(country)
    totalDatos, totalMeses, total = pais(country,groupData)
    print(totalDatos,totalMeses)
    return jsonify({'totalDatos': totalDatos.to_json(), 'totalMeses': totalMeses.to_json(), 'totalCasos' : total})
    #return 'ds'
    
if __name__ == "__main__":
    
    app.config.from_object(Config())

    # it is also possible to enable the API directly
    # scheduler.api_enabled = True
    scheduler.init_app(app)
    scheduler.start()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Oct/2020 17:02:02] "OPTIONS //postDataCountry HTTP/1.1" 200 -


Canada


<ipython-input-3-be1420abd8d8>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
127.0.0.1 - - [07/Oct/2020 17:02:02] "POST //postDataCountry HTTP/1.1" 200 -


  fechaTotal  datosTotal
0    January           4
1   February          20
2      March        8527
3      April       54457
4        May       92479
5       June      106097
6       July      118281
7     August      130918
8  September      161107
9    October      173756    MesDate    fecha  datos
0    April   4/1/20   9560
1    April   4/2/20  11284
2    April   4/3/20  12437
3    April   4/4/20  12978
4    April   4/5/20  15756
..     ...      ...    ...
56     May  5/27/20  88989
57     May  5/28/20  89976
58     May  5/29/20  90909
59     May  5/30/20  91681
60     May  5/31/20  92479

[61 rows x 3 columns]


...

  fechaTotal  datosTotal
0    January           4
1   February          20
2      March        8527
3      April       54457
4        May       92479
5       June      106097
6       July      118281
7     August      130918
8  September      161107
9    October      173756    MesDate    fecha  datos
0    April   4/1/20   9560
1    April   4/2/20  11284
2    April   4/3/20  12437
3    April   4/4/20  12978
4    April   4/5/20  15756
..     ...      ...    ...
56     May  5/27/20  88989
57     May  5/28/20  89976
58     May  5/29/20  90909
59     May  5/30/20  91681
60     May  5/31/20  92479

[61 rows x 3 columns]


127.0.0.1 - - [07/Oct/2020 19:55:06] "OPTIONS //postDataCountry HTTP/1.1" 200 -


Canada


<ipython-input-3-be1420abd8d8>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
127.0.0.1 - - [07/Oct/2020 19:55:06] "POST //postDataCountry HTTP/1.1" 200 -


  fechaTotal  datosTotal
0    January           4
1   February          20
2      March        8527
3      April       54457
4        May       92479
5       June      106097
6       July      118281
7     August      130918
8  September      161107
9    October      173756    MesDate    fecha  datos
0    April   4/1/20   9560
1    April   4/2/20  11284
2    April   4/3/20  12437
3    April   4/4/20  12978
4    April   4/5/20  15756
..     ...      ...    ...
56     May  5/27/20  88989
57     May  5/28/20  89976
58     May  5/29/20  90909
59     May  5/30/20  91681
60     May  5/31/20  92479

[61 rows x 3 columns]
